In [111]:
import pyodbc
from sqlalchemy import create_engine

import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format
#pd.reset_option('display.float_format')

from google.oauth2.service_account import Credentials

In [112]:
credentials = Credentials.from_service_account_file(r"C:\Users\jesus\.gc\sien-project.json")

In [113]:
server_name = "jesus"
database_name = "DWH_SUPERMAYORISTA"

engine =create_engine(f"mssql+pyodbc://@{server_name}/{database_name}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server",
                     fast_executemany=True)

In [114]:
engine.connect()

### DIM_BANCO

In [115]:
dim_banco=pd.read_sql_table('DIM_BANCO', engine)  

In [116]:
display(dim_banco)
print(dim_banco.shape)

,cod_banco,nombre_banco,comision
0,BAN01,BBVA,2.80
1,BAN02,BCP,3.50
2,BAN03,SCOTIABANK,2.00
3,BAN04,INTERBANK,3.00
4,BAN05,MIBANCO,4.20
5,BAN06,CAJA HUANCAYO,3.30


(6, 3)


### DIM_CLIENTE

In [117]:
dim_cliente=pd.read_sql_table('DIM_CLIENTE', engine)  

In [118]:
display(dim_cliente.head())
print(dim_cliente.shape)

,dni_cliente,sexo_cliente,fecha_nacimiento,departamento_cliente,distrito_cliente,ubigeo_cliente,punto_geografico
0,10001165,FEMENINO,1998-04-30,HUANUCO,JACAS GRANDE,100504,"-9.51519574367, -76.6684109766"
1,10001400,MASCULINO,1995-11-25,PASCO,PALCAZU,190304,"-10.2343589434, -75.2761411859"
2,10002816,MASCULINO,1983-03-19,AREQUIPA,YANAQUIHUA,40608,"-15.701962903, -73.0291096848"
3,10004947,MASCULINO,1973-08-26,PUNO,YANAHUAYA,211208,"-14.3311418177, -69.1270554713"
4,10005087,FEMENINO,1975-05-26,LIMA,ZUÑIGA,150516,"-12.8298665497, -76.0140629517"


(50282, 7)


### DIM_PRODUCTO

In [119]:
dim_producto=pd.read_sql_table('DIM_PRODUCTO', engine)  

In [120]:
display(dim_producto.head())
print(dim_producto.shape)

,cod_producto,nombre_producto,categoria_producto,subcategoria_producto,precio_unitario
0,PROD00001,LIMPIA TAPICERÍA EN ESPUMA BINNER,ASEO DE HOGAR,ALFOMBRAS,779.50
1,PROD00002,LIMPIA TAPICERÍA Y ALFOMBRAS BINNER,ASEO DE HOGAR,ALFOMBRAS,699.65
2,PROD00003,LIMPIADOR DR. BECKMANN ALFOMBRAS 2EN1 PODER OXI,ASEO DE HOGAR,ALFOMBRAS,825.65
3,PROD00004,ACEITE BON AIRE ELÉCTRICO PERFUME FLORES,ASEO DE HOGAR,AMBIENTADORES,749.50
4,PROD00005,AMBIENTADOR AIRE VARITAS BAMBÚ BONAIRE 2 REPUE...,ASEO DE HOGAR,AMBIENTADORES,"1,749.50"


(6624, 5)


### DIM_VENDEDOR

In [121]:
dim_vendedor=pd.read_sql_table('DIM_VENDEDOR', engine)  

In [122]:
display(dim_vendedor.head())
print(dim_vendedor.shape)

,cod_vendedor,vendedor,supervisor
0,VEND001,"MARTINEZ DIAZ, RODRIGO",None
1,VEND002,"RODRIGO ALVAREZ, RICARDO",VEND001
2,VEND003,"ALVARO MEDINA, ANGELA",VEND001
3,VEND004,"PALACIOS MOJICO, ALONSO",VEND001
4,VEND005,"SUAREZ NORTEL, FABIOLA",VEND001


(56, 3)


### FACT_VENTA

In [123]:
fact_venta=pd.read_sql_table('FACT_VENTA', engine)  
fact_venta['cantidad']=fact_venta['cantidad'].astype('Int64')

In [124]:
display(fact_venta.head())
print(fact_venta.shape)

,cod_documento,dni_cliente,vendedor,nombre_tipo_documento,estado,fecha_venta,cod_producto,precio_unitario,cantidad,cod_banco
0,B000038653,10001165,"RONDON SANDOVAL, ELIZABETH",BOLETA,VENDIDO,2020-03-16,PROD03123,249.50,9,None
1,B000038653,10001165,"RONDON SANDOVAL, ELIZABETH",BOLETA,VENDIDO,2020-03-16,PROD01504,524.65,11,None
2,B000002024,10001400,"SOLIS VEGA, BRITANY",BOLETA,VENDIDO,2018-02-13,PROD05231,363.65,10,None
3,B000002024,10001400,"SOLIS VEGA, BRITANY",BOLETA,VENDIDO,2018-02-13,PROD03565,519.50,10,None
4,B000002024,10001400,"SOLIS VEGA, BRITANY",BOLETA,VENDIDO,2018-02-13,PROD04384,209.65,13,None


(434161, 10)


### 1. Total de ventas por periodo

In [75]:
# Crear una copia de fact_venta
df1=fact_venta.copy()

# 1.Añadir columna Total_Bruto 
# 2.Añadir columna Total_Neto
df1['total_bruto'] = df1['precio_unitario']*df1['cantidad']
df1['total_neto'] = df1['nombre_tipo_documento'].apply(lambda x: 1 if x == "BOLETA" else 1.18)*df1['total_bruto']

# 1.Agrupar la suma de Total_Bruto y Total_Neto por fecha_venta
# 2.Eliminar el indice creado en el punto 1
df1_agrupado=df1.groupby(df1['fecha_venta'].dt.to_period("M"))[['total_bruto','total_neto']].sum()\
                .reset_index(level=[0])

# Cambiamos de formato a la columna fecha_venta de (Año y Mes) a (Año, Mes y Día)
df1_agrupado['fecha_venta'] = df1_agrupado['fecha_venta'].astype(str)
df1_agrupado['fecha_venta'] = pd.to_datetime(df1_agrupado['fecha_venta'])

# Cambiamos nombre de las columnas
df1_agrupado.rename(columns={"fecha_venta": "periodo"},inplace=True)

In [76]:
display(df1_agrupado.head())
print(df1_agrupado.shape)

,periodo,total_bruto,total_neto
0,2018-01-01,"26,256,892.70","26,256,892.70"
1,2018-02-01,"20,028,813.15","20,028,813.15"
2,2018-03-01,"24,861,858.10","24,861,858.10"
3,2018-04-01,"23,111,322.95","23,111,322.95"
4,2018-05-01,"24,853,712.65","24,853,712.65"


(59, 3)


In [77]:
# Definir tabla en Big Query
target_table = 'dwh_queries.Total_Ventas'
project_id = 'sien-project'
df1_agrupado.to_gbq(target_table, 
                    project_id,
                    if_exists='replace',
                    credentials=credentials)

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 499.50it/s]


### 2. Top N productos más vendidos (categoría, subcategoría)

In [86]:
# Crear una copia de fact_venta
df2=fact_venta.copy()

# 1.Agrupar la suma de cantidad por fecha_venta y cod_producto
# 2.Ordenar fecha_venta,cantidad de forma ascendente, descendente respectivamente
# 3.Tomar los 10 primeros valores segun fecha_venta
# 4.Eliminar el indice creado en el punto 1
# 5.Hacer un left join con cod_producto
df2_query=df2.groupby([df2['fecha_venta'].dt.to_period("M"),'cod_producto'])[['cantidad']].sum()\
            .sort_values(by=['fecha_venta','cantidad'],ascending=[True,False])\
            .groupby(['fecha_venta']).head(10)\
            .reset_index(level=[0,1])\
            .merge(dim_producto,how='left',on='cod_producto')

# Cambiamos de formato a la columna fecha_venta de (Año y Mes) a (Año, Mes y Día)
df2_query['fecha_venta'] = df2_query['fecha_venta'].astype(str)
df2_query['fecha_venta'] = pd.to_datetime(df2_query['fecha_venta'])

# Cambiamos nombre de las columnas
df2_query.rename(columns={"fecha_venta": "periodo"},inplace=True)

In [87]:
display(df2_query.head())
print(df2_query.shape)

,periodo,cod_producto,cantidad,nombre_producto,categoria_producto,subcategoria_producto,precio_unitario
0,2018-01-01,PROD01212,55,SOMBRAS SÚPER FANTASTIC TRIO TONO 01 PRIMAVERA...,BELLEZA,MAQUILLAJE,251.65
1,2018-01-01,PROD05624,53,CERVEZA RUBIA ERDINGER BOTELLA,VINOS Y LICORES,CERVEZAS,699.50
2,2018-01-01,PROD02715,51,EMPANIZADOR DE ARROZ NNOVA RICE,DESPENSA,HARINAS Y MEZCLAS PARA PREPARAR,199.50
3,2018-01-01,PROD03750,43,ALPINETTE CEREZA,"LÁCTEOS, HUEVOS Y REFRIGERADOS",POSTRES REFRIGERADOS,134.50
4,2018-01-01,PROD04121,43,CEPILLO RESTREGÓN MULTIUSOS ETERNA,LIMPIEZA DE COCINA,"BALDES, ESCOBAS Y ACCESORIOS",214.50


(590, 7)


In [90]:
# Definir tabla en Big Query
target_table = 'dwh_queries.Top_Productos'
project_id = 'sien-project'
df2_query.to_gbq(target_table, 
                project_id,
                if_exists='replace',
                credentials=credentials)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


### 3. Top N vendedores más destacados

In [65]:
# Crear una copia de fact_venta
df3=fact_venta.copy()

# Añadir columna Total_Bruto 
df3['total_bruto']=df3['precio_unitario']*df3['cantidad']

# 1.Agrupar la suma de total_bruto por fecha_venta
# 2.Ordenar fecha_venta,total_bruto de forma ascendente, descendente respectivamente
# 3.Tomar los 10 primeros valores segun fecha_venta
# 4.Eliminar el indice creado en el punto 1
# 5.Hacer un left join con dim_vendedor
df3_query=df3.groupby([df3['fecha_venta'].dt.to_period("M"),'vendedor'])[['total_bruto']].sum()\
            .sort_values(by=['fecha_venta','total_bruto'],ascending=[True,False])\
            .groupby(['fecha_venta']).head(10)\
            .reset_index(level=[0,1])\
            .merge(dim_vendedor,how='left',on='vendedor')

# Cambiamos de formato a la columna fecha_venta de (Año y Mes) a (Año, Mes y Día)
df3_query['fecha_venta'] = df3_query['fecha_venta'].astype(str)
df3_query['fecha_venta'] = pd.to_datetime(df3_query['fecha_venta'])
# Añadimos columna si es supervisor
df3_query['is_supervisor']=df3_query[['supervisor']].apply(lambda x: 'Supervisor' if x['supervisor']==None else 'Vendedor', axis=1)
# Añadimos columna 'pregunta' para unir con la query 4
df3_query['pregunta']='Query_3'

# Cambiamos nombre de las columnas
df3_query.rename(columns={"fecha_venta": "periodo"},inplace=True)


In [66]:
display(df3_query.head())
print(df3_query.shape)

,periodo,vendedor,total_bruto,cod_vendedor,supervisor,is_supervisor,pregunta
0,2018-01-01,"ZAPATA LOYO, NICOLE","827,214.30",VEND051,VEND049,Vendedor,Query_3
1,2018-01-01,"VELARDO ALVARADO, MAITE","815,223.60",VEND010,VEND009,Vendedor,Query_3
2,2018-01-01,"ALCOCER INGUNZA, LIZETH","763,384.90",VEND040,VEND033,Vendedor,Query_3
3,2018-01-01,"GUERRERO CIEZA, JACKELINE","723,392.65",VEND048,VEND041,Vendedor,Query_3
4,2018-01-01,"POMA CANTO, MARCOS","681,805.05",VEND031,VEND025,Vendedor,Query_3


(590, 7)


### 4. Top supervisores con baja efectividad

In [80]:
# Crear una copia de fact_venta
df4=fact_venta.copy()

# Añadir columna total_bruto 
df4['total_bruto']=df4['precio_unitario']*df4['cantidad']

# 1.Agrupar la suma de total_bruto por fecha_venta y vendedor
# 2.Ordenar fecha_venta,total_bruto de forma ascendente, descendente respectivamente
# 3.Eliminar el indice creado en el punto 1
df4_query=df4.groupby([df4['fecha_venta'].dt.to_period("M"),'vendedor'])[['total_bruto']].sum()\
            .sort_values(by=['fecha_venta','total_bruto'],ascending=[True,False])\
            .reset_index(level=[0,1])

# Crear una copia de dim_vendedor
df4_vendedor=dim_vendedor.copy()
# Modificar columna supervisor para que los valores nulos sean reemplazados por el valor de cod_vendedor
df4_vendedor['supervisor']=df4_vendedor[['supervisor','cod_vendedor']]\
                                .apply(lambda x: x['cod_vendedor'] if x['supervisor']==None else x['supervisor'],axis=1)

# 1.Hacer un left join de df4_query con df4_vendedor
# 2.Ordenar fecha_venta,cod_vendedor de forma ascendente, ascendente respectivamente
# 3.Eliminar el indice al ordenar en el punto 2
df_4_p=df4_query.merge(df4_vendedor,how='left',on='vendedor')\
        .sort_values(by=['fecha_venta','cod_vendedor'],ascending=[True,True])\
        .reset_index(drop=True)

# 1.Agrupar la suma de total_bruto por fecha_venta y supervisor
# 2.Ordenar fecha_venta,total_bruto de forma ascendente, ascendente respectivamente
# 3.Tomar los 5 primeros valores segun fecha_venta
# 4.Eliminar el indice creado en el punto 1
# 5.Cambiar nombre de las columnas
# 6.Hacer un left join con df4_vendedor
df_4_min=df_4_p.groupby(['fecha_venta','supervisor'])[['total_bruto']].sum()\
            .sort_values(by=['fecha_venta','total_bruto'],ascending=[True,True])\
            .groupby(['fecha_venta']).head(7)\
            .reset_index(level=[0,1])\
            .rename(columns={"supervisor":"cod_vendedor"})\
            .merge(df4_vendedor,how='left',on='cod_vendedor')

# Cambiamos de formato a la columna fecha_venta de (Año y Mes) a (Año, Mes y Día)
df_4_min['fecha_venta'] = df_4_min['fecha_venta'].astype(str)
df_4_min['fecha_venta'] = pd.to_datetime(df_4_min['fecha_venta'])

# Cambiamos nombre de las columnas
df_4_min.rename(columns={"fecha_venta": "periodo"},inplace=True)

# Añadimos columna 'pregunta' para unir con la query 3
df_4_min['pregunta']='Query_4'
# Añadimos columna is_supervvisor
df_4_min['is_supervisor']='Supervisor'

In [82]:
# Union query 3 con query 4
df_union=pd.concat([df3_query, df_4_min])

display(df_union.head())
print(df_union.shape)

,periodo,vendedor,total_bruto,cod_vendedor,supervisor,is_supervisor,pregunta
0,2018-01-01,"ZAPATA LOYO, NICOLE","827,214.30",VEND051,VEND049,Vendedor,Query_3
1,2018-01-01,"VELARDO ALVARADO, MAITE","815,223.60",VEND010,VEND009,Vendedor,Query_3
2,2018-01-01,"ALCOCER INGUNZA, LIZETH","763,384.90",VEND040,VEND033,Vendedor,Query_3
3,2018-01-01,"GUERRERO CIEZA, JACKELINE","723,392.65",VEND048,VEND041,Vendedor,Query_3
4,2018-01-01,"POMA CANTO, MARCOS","681,805.05",VEND031,VEND025,Vendedor,Query_3


(1003, 7)


In [83]:
# Definir tabla en Big Query
target_table = 'dwh_queries.Top_Vendedores_Bottom_Supervisores'
project_id = 'sien-project'
df_union.to_gbq(target_table, 
                project_id,
                if_exists='replace',
                credentials=credentials)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


### 5. Top N zonas con mayor venta (Departamento y Distrito)

In [108]:
# Crear una copia de fact_venta
df5=fact_venta.copy()\
    .merge(dim_cliente, how='left',on='dni_cliente' )

# Añadir columna total_bruto 
df5['total_bruto']=df5['precio_unitario']*df5['cantidad']

# 1.Agrupar la suma de total_bruto por fecha_venta,departamento_cliente,ubigeo_cliente y punto_geografico
# 2.Ordenar fecha_venta,total_bruto de forma ascendente, descendente respectivamente
# 3.Tomar los 10 primeros valores segun fecha_venta
# 4.Eliminar el indice creado en el punto 1
df5_v=df5.groupby([df5['fecha_venta'].dt.to_period("M"),'departamento_cliente','distrito_cliente','ubigeo_cliente',
                     'punto_geografico'])[['total_bruto']].sum()\
        .sort_values(by=['fecha_venta','total_bruto'],ascending=[True,False])\
        .groupby(['fecha_venta']).head(10)\
        .reset_index(level=[0,1,2,3,4])

# Cambiamos de formato a la columna fecha_venta de (Año y Mes) a (Año, Mes y Día)
df5_v['fecha_venta'] = df5_v['fecha_venta'].astype(str)
df5_v['fecha_venta'] = pd.to_datetime(df5_v['fecha_venta'])

# Cambiamos nombre de las columnas
df5_v.rename(columns={"fecha_venta": "periodo","departamento_cliente":"departamento",
                     "distrito_cliente":"distrito","ubigeo_cliente":"cliente"},inplace=True)

In [109]:
display(df5_v.head())
print(df5_v.shape)

,periodo,departamento,distrito,cliente,punto_geografico,total_bruto
0,2018-01-01,SAN MARTIN,CACATACHI,220903,"-6.46781171215, -76.4384114549","227,089.30"
1,2018-01-01,APURIMAC,JUAN ESPINOZA MEDRANO,30304,"-14.5586108444, -72.9083883011","226,038.50"
2,2018-01-01,PIURA,CANCHAQUE,200302,"-5.31206763234, -79.6258326484","195,181.90"
3,2018-01-01,PUNO,YANAHUAYA,211208,"-14.3311418177, -69.1270554713","193,927.00"
4,2018-01-01,LIMA,COPA,150302,"-10.3644795714, -77.0233787817","190,143.25"


(590, 6)


In [110]:
# Definir tabla en Big Query
target_table = 'dwh_queries.Top_Zonas'
project_id = 'sien-project'
df5_v.to_gbq(target_table, 
             project_id,
             if_exists='replace',
             credentials=credentials)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


### 6. Banco más factible para solicitar un ajuste de comisión

In [128]:
# Crear una copia de fact_venta
df6=fact_venta.copy()\
    .merge(dim_banco, how='left', on='cod_banco')

# 1.Agrupar la suma de comision por fecha_venta y nombre_banco
# 2.Ordenar fecha_venta,comision de forma ascendente, descendente respectivamente
# 3.Tomar los 6 primeros valores segun fecha_venta
# 4.Eliminar el indice creado en el punto 1
df6_final=df6.groupby([df6['fecha_venta'].dt.to_period("M"),'nombre_banco'])[['comision']].sum()\
            .sort_values(by=['fecha_venta','comision'],ascending=[True,False])\
            .groupby(['fecha_venta']).head(6)\
            .reset_index(level=[0,1])

# Cambiamos de formato a la columna fecha_venta de (Año y Mes) a (Año, Mes y Día)
df6_final['fecha_venta'] = df6_final['fecha_venta'].astype(str)
df6_final['fecha_venta'] = pd.to_datetime(df6_final['fecha_venta'])

# Cambiamos nombre de las columnas
df6_final.rename(columns={"fecha_venta": "periodo","nombre_banco":"banco"},inplace=True)

In [132]:
display(df6_final.head())
print(df6_final.shape)

,periodo,banco,comision
0,2022-01-01,MIBANCO,"8,366.40"
1,2022-01-01,BCP,"7,077.00"
2,2022-01-01,INTERBANK,"6,081.00"
3,2022-01-01,BBVA,"5,796.00"
4,2022-01-01,CAJA HUANCAYO,"5,593.50"


(66, 3)


In [130]:
# Definir tabla en Big Query
target_table = 'dwh_queries.Top_Bancos'
project_id = 'sien-project'
df6_final.to_gbq(target_table, 
             project_id,
             if_exists='replace',
             credentials=credentials)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


### 7. Top N vendedores que más comisionan por venta

In [141]:
# Crear una copia de fact_venta
df7=fact_venta.copy()
df7['total_bruto']=df7['precio_unitario']*df7['cantidad']

# 1.Agrupar la suma de total_bruto por fecha_venta, nombre_tipo_documento y vendedor
# 2.Ordenar fecha_venta,nombre_tipo_documento y total_bruto de forma ascendente,ascendente y  descendente respectivamente
# 3.Eliminar el indice creado en el punto 1
df7_total=df7.groupby(['fecha_venta','nombre_tipo_documento','vendedor'])[['total_bruto']].sum()\
            .sort_values(by=['fecha_venta','nombre_tipo_documento','total_bruto'],ascending=[True,True,False])\
            .reset_index(level=[0,1,2])

# Agregar columnas comision_1 y comision_2
df7_total['comision_1']=df7_total['nombre_tipo_documento'].apply(lambda x: 0.012 if x=='BOLETA' else 0.018)*df7_total['total_bruto']
df7_total['comision_2']=df7_total[['nombre_tipo_documento','total_bruto']]\
                                .apply(lambda x: 0.01*x['total_bruto'] if (x['total_bruto']>20000 and x['nombre_tipo_documento']=='FACTURA') else 0.0,axis=1)

# Crear una copia de dim_vendedor
df7_vendedor=dim_vendedor.copy()
# Modificar columna supervisor para que los valores nulos sean reemplazados por el valor de cod_vendedor
df7_vendedor['supervisor']=df7_vendedor[['supervisor','cod_vendedor']]\
                                .apply(lambda x: x['cod_vendedor'] if x['supervisor']==None else x['supervisor'],axis=1)

# 1.Agrupar la suma de comision_1 y comision_2 por fecha_venta, nombre_tipo_documento y vendedor
# 2.Eliminar el indice creado en el punto 1
# 3.Hacer un left join con df7_vendedor
# 4.Ordenar fecha_venta,nombre_tipo_documento y cod_vendedor de forma ascendente, ascendente y descendente respectivamente
# 5.Eliminar el indice y resetearlo
df7_d=df7_total.groupby([df7_total['fecha_venta'].dt.to_period("M"),'nombre_tipo_documento','vendedor'])[['comision_1','comision_2']].sum()\
        .reset_index(level=[0,1,2])\
        .merge(df7_vendedor, how='left', on='vendedor')\
        .sort_values(by=['fecha_venta','nombre_tipo_documento','cod_vendedor'],ascending=[True,True,True])\
        .reset_index(drop=True)

# Crear columna comision_supervisor, para realizar sumatoria de comision_2 por particion fecha_enta y supervisor
df7_d['comision_supervisor'] = df7_d.groupby(['fecha_venta','supervisor'])[['comision_2']].transform(np.sum)

# Crear columna comision_total, para visualizar la comision que se lleva cada vendedor
df7_d['comision_total']=df7_d[['cod_vendedor','supervisor','comision_1','comision_2','comision_supervisor']]\
    .apply(lambda x: (x['comision_1']+x['comision_supervisor']-x['comision_2']) if (x['cod_vendedor']==x['supervisor']) else x['comision_1'],axis=1)

# 1.Agrupar la suma de comision_1 por fecha_venta, nombre_tipo_documento y vendedor
# 2.Ordenar fecha_venta y comision_total de forma ascendente y descendente respectivamente
# 3.Tomar los 10 primeros valores segun fecha_venta
# 4.Eliminar el indice creado en el punto 1
# 5.Hacer un left join con df7_vendedor
df7_e=df_7_d.groupby(['fecha_venta','cod_vendedor'])[['comision_total']].sum()\
        .sort_values(by=['fecha_venta','comision_total'],ascending=[True,False])\
        .groupby(['fecha_venta']).head(10)\
        .reset_index(level=[0,1])\
        .merge(df7_vendedor,on='cod_vendedor',how='left')

# Cambiamos de formato a la columna fecha_venta de (Año y Mes) a (Año, Mes y Día)
df7_e['fecha_venta'] = df7_e['fecha_venta'].astype(str)
df7_e['fecha_venta'] = pd.to_datetime(df7_e['fecha_venta'])

# Añadimos columna si es supervisor
df7_e['is_supervisor']=df7_e[['supervisor','cod_vendedor']].apply(lambda x: 'Supervisor' if x['cod_vendedor']==x['supervisor'] else 'Vendedor', axis=1)

# Cambiamos nombre de las columnas
df7_e.rename(columns={"fecha_venta": "periodo"},inplace=True)

In [148]:
display(df7_e.head())
print(df7_e.shape)

,periodo,cod_vendedor,comision_total,vendedor,supervisor,is_supervisor
0,2018-01-01,VEND051,"9,926.57","ZAPATA LOYO, NICOLE",VEND049,Vendedor
1,2018-01-01,VEND010,"9,782.68","VELARDO ALVARADO, MAITE",VEND009,Vendedor
2,2018-01-01,VEND040,"9,160.62","ALCOCER INGUNZA, LIZETH",VEND033,Vendedor
3,2018-01-01,VEND048,"8,680.71","GUERRERO CIEZA, JACKELINE",VEND041,Vendedor
4,2018-01-01,VEND031,"8,181.66","POMA CANTO, MARCOS",VEND025,Vendedor


(590, 6)


In [147]:
# Definir tabla en Big Query
target_table = 'dwh_queries.Top_Vendedores_Comision'
project_id = 'sien-project'
df7_e.to_gbq(target_table, 
             project_id,
             if_exists='replace',
             credentials=credentials)

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 989.22it/s]


# Referencias

Pandas - Save DataFrame to BigQuery

https://kontext.tech/article/682/pandas-save-dataframe-to-bigquery
